In [18]:
#! pip install sqlalchemy

In [19]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text
from sqlalchemy.sql import func, distinct, exists
import pandas as pd

In [20]:
engine = create_engine('sqlite:///../Data/Students.db')
metadata_obj = MetaData() # Объект-контейнер, содержит информацию о схеме данных
metadata_obj.reflect(bind=engine) # Отображение БД

Создание объектов таблиц

In [21]:
exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")

In [22]:
conn=engine.connect() # Соединение

<img src=../Img/pencil.png style="width:190px;height:190px">
<p style="background-color:powderblue; font-size:125%; color: rebeccapurple;">Выполните задания</p>
<p>Выполните задания с помощью соедиинения таблиц в Jupyter Notebook </p>

1. Составьте список названий университетов с указанием суммарного количества аудиторных часов на изучение каждого предмета.

2. Получите список названий университетов вместе с фамилиями и именами студентов, получающих максимальную для каждого университета стипендию.

3. Напишите запрос для получения списка преподавателей, преподающих только один предмет. Отсортируйте результат по фамилии преподавателя. Результат должен содержать имя и фамилию преподавателя, название предмета.

4. Напишите запрос для получения списка университетов вместе с фамилиями студентов, получивших хотя бы одну отличную оценку. Результат отсортируйте по названию университета.

5. Напишите запрос, выполняющий вывод имен и фамилий преподавателей, проводящих занятия на первом курсе. Отсортируйте результат по фаимилии преподавателя.

6. Напишите запрос, выполняющий вывод наименований предметов обучения, читаемых двумя или более преподавателями. Результат должен состоять из одного столбца названий предметов, без повторов, в алфавитном порядке по возрастанию.

# Решение

In [23]:
def print_result(s):
    print(s, end = '\n\n')
    res = conn.execute(s).fetchall()
    print(len(res))
    return res

In [24]:
u = university.alias('u')
st = student.alias('st')
l = lecturer.alias('l')
sl = subj_lect.alias('sl')
su = subject.alias('su')
em = exam_marks.alias('em')
st2 = student.alias('st2')

In [25]:
#1. Составьте список названий университетов с указанием суммарного количества аудиторных часов на изучение каждого предмета.


s = select(u.c.univ_name, su.c.subj_name, func.sum(su.c.hour))
s = s.select_from(u)
s = s.join(l)
s = s.join(sl)
s = s.join(su)
s = s.group_by(u.c.univ_name, su.c.subj_name)

print_result(s)

SELECT u.univ_name, su.subj_name, sum(su.hour) AS sum_1 
FROM university AS u JOIN lecturer AS l ON u.univ_id = l.univ_id JOIN subj_lect AS sl ON l.lecturer_id = sl.lecturer_id JOIN subject AS su ON su.subj_id = sl.subj_id GROUP BY u.univ_name, su.subj_name

22


[('ВГУ', 'Анализ данных', 42),
 ('ВГУ', 'История', 68),
 ('ВГУ', 'ОБЖ', 34),
 ('ВГУ', 'Программирование на языках Python и SQL', 34),
 ('ВГУ', 'Теория вероятностей и математическая статистика', 72),
 ('ВГУ', 'Физика', 34),
 ('КФУ', 'Информатика', 56),
 ('КФУ', 'История', 34),
 ('КФУ', 'Математика', 56),
 ('МГТУ им. Баумана', 'Английский язык', 56),
 ('МГТУ им. Баумана', 'Программирование на языках Python и SQL', 34),
 ('МГУ', 'Английский язык', 56),
 ('МГУ', 'Информатика', 56),
 ('МГУ', 'ОБЖ', 34),
 ('НГУ', 'Программирование на языках Python и SQL', 34),
 ('РГУ', 'Информатика', 56),
 ('РГУ', 'Программирование на языках Python и SQL', 34),
 ('ТГУ', 'Информатика', 112),
 ('ТГУ', 'Физкультура', 34),
 ('УЛГУ', 'ОБЖ', 34),
 ('ФГБОУ "Политех"', 'Английский язык', 56),
 ('ФГБОУ "Политех"', 'Программирование на языках Python и SQL', 34)]

In [26]:
# 2. Получите список названий университетов вместе с фамилиями и именами студентов, 
# получающих максимальную для каждого университета стипендию.


subq = select(func.max(st2.c.stipend))
subq = subq.where(st2.c.univ_id == u.c.univ_id).scalar_subquery() # когда единственное, берем скаляр

s = select(u.c.univ_name, st.c.surname + ' ' + st.c.name, st.c.stipend)
s = s.select_from(u)
s = s.join(st)
s = s.where(st.c.stipend == subq)

print_result(s)

SELECT u.univ_name, st.surname || :surname_1 || st.name AS anon_1, st.stipend 
FROM university AS u JOIN student AS st ON u.univ_id = st.univ_id 
WHERE st.stipend = (SELECT max(st2.stipend) AS max_1 
FROM student AS st2 
WHERE st2.univ_id = u.univ_id)

24


[('ВГУ', 'Зайцева Ольга', 250),
 ('ВГУ', 'Белкин Вадим', 250),
 ('КФУ', 'Жук Оксана', 250),
 ('ВГУ', 'Некрасова Вероника', 250),
 ('ФинУ', 'Казакова Ольга', 250),
 ('ФГБОУ "Политех"', 'Миронова Анна', 200),
 ('ВГМА', 'Кондрат Синицин', 200),
 ('УЛГУ', 'Пономарева Алина', 250),
 ('ВГМА', 'Валерия Овчинникова', 200),
 ('МГУ', 'Блохина Ираида', 250),
 ('ТГУ', 'Мельникова Светлана', 250),
 ('ФГБОУ "Политех"', 'Вишнякова Анжела', 200),
 ('НГУ', 'Трофимов Владислав', 250),
 ('ФинУ', 'Морозов Кирилл', 250),
 ('БГУ', 'Суханов Константин', 250),
 ('РГУ', 'Панов Арсений', 250),
 ('РГУ', 'Кудряшов Роман', 250),
 ('ВГМА', 'Журавлев Фома', 200),
 ('МГУ', 'Ершов Митрофан', 250),
 ('КФУ', 'Русаков Давид', 250),
 ('БГУ', 'Вячеслав Пахомов', 250),
 ('БГУ', 'Петрова Екатерина', 250),
 ('ФинУ', 'Петров Вадим', 250),
 ('МГТУ им. Баумана', 'Миронова Федор', 200)]

In [27]:
# 3. Напишите запрос для получения списка преподавателей, преподающих только один предмет. 
# Отсортируйте результат по фамилии преподавателя. 
# Результат должен содержать имя и фамилию преподавателя, название предмета.


s = select(l.c.surname + ' ' + l.c.name)
s = s.select_from(l)
s = s.join(sl)
s = s.group_by(l.c.surname + ' ' + l.c.name)
s = s.having(func.count(sl.c.subj_id) == 1)

print_result(s)

SELECT l.surname || :surname_1 || l.name AS anon_1 
FROM lecturer AS l JOIN subj_lect AS sl ON l.lecturer_id = sl.lecturer_id GROUP BY l.surname || :surname_2 || l.name 
HAVING count(sl.subj_id) = :count_1

12


[('Алексеев Елисей',),
 ('Беляков Мефодий',),
 ('Брагина Нинель',),
 ('Дорофеев Прокофий',),
 ('Зимин Юлиан',),
 ('Колесников Борис',),
 ('Кудрявцева Анна',),
 ('Лагутин Павел',),
 ('Мамонтова Акулина',),
 ('Никонов Иван',),
 ('Сафонова Лукия',),
 ('Тихонов Ладомир',)]

In [28]:
# 4. Напишите запрос для получения списка университетов вместе с фамилиями студентов, 
# получивших хотя бы одну отличную оценку. 
# Результат отсортируйте по названию университета.


s = select(u.c.univ_name, st.c.surname + ' ' + st.c.name).distinct()
s = s.where(em.c.mark == 5)
s = s.select_from(u)
s = s.join(st)
s = s.join(em)
s = s.group_by(u.c.univ_name, st.c.surname + ' ' + st.c.name)

print_result(s)

SELECT DISTINCT u.univ_name, st.surname || :surname_1 || st.name AS anon_1 
FROM university AS u JOIN student AS st ON u.univ_id = st.univ_id JOIN exam_marks AS em ON st.student_id = em.student_id 
WHERE em.mark = :mark_1 GROUP BY u.univ_name, st.surname || :surname_2 || st.name

10


[('ВГУ', 'Белкин Вадим'),
 ('ВГУ', 'Борисова Марина'),
 ('ВГУ', 'Жданова Ирина'),
 ('ВГУ', 'Зайцева Ольга'),
 ('ВГУ', 'Кузнецов Борис'),
 ('МГТУ им. Баумана', 'Лапина Элеонора'),
 ('МГУ', 'Блохина Ираида'),
 ('ТГУ', 'Мельникова Светлана'),
 ('ТГУ', 'Пак Ирина'),
 ('ФГБОУ "Политех"', 'Васильев Матвей')]

In [29]:
# 5. Напишите запрос, выполняющий вывод имен и фамилий преподавателей, 
# проводящих занятия на первом курсе. Отсортируйте результат по фаимилии преподавателя.


s = select(l.c.surname + ' ' + l.c.name)
s = s.where(su.c.semester < 3)
s = s.select_from(l)
s = s.join(sl)
s = s.join(su)
s = s.group_by(l.c.surname + ' ' + l.c.name)

print_result(s)

SELECT l.surname || :surname_1 || l.name AS anon_1 
FROM lecturer AS l JOIN subj_lect AS sl ON l.lecturer_id = sl.lecturer_id JOIN subject AS su ON su.subj_id = sl.subj_id 
WHERE su.semester < :semester_1 GROUP BY l.surname || :surname_2 || l.name

14


[('Алексеев Елисей',),
 ('Беляков Мефодий',),
 ('Беспалов Юлиан',),
 ('Брагина Нинель',),
 ('Волков Михаил',),
 ('Дорофеев Прокофий',),
 ('Зимин Юлиан',),
 ('Колесников Борис',),
 ('Кудрявцева Анна',),
 ('Куликова Ираида',),
 ('Максимова Иванна',),
 ('Мамонтова Акулина',),
 ('Струков Николай',),
 ('Тихонов Ладомир',)]

In [30]:
# 6. Напишите запрос, выполняющий вывод наименований предметов обучения, 
# читаемых двумя или более преподавателями. Результат должен состоять из одного столбца названий предметов, 
# без повторов, в алфавитном порядке по возрастанию.


s = select(su.c.subj_name)
s = s.select_from(su)
s = s.join(sl)
s = s.join(l)
s = s.group_by(su.c.subj_name)
s = s.having(func.count(sl.c.lecturer_id) > 1) # в where нельзя использовать функции агрегации
s = s.order_by(su.c.subj_name)

print_result(s)

SELECT su.subj_name 
FROM subject AS su JOIN subj_lect AS sl ON su.subj_id = sl.subj_id JOIN lecturer AS l ON l.lecturer_id = sl.lecturer_id GROUP BY su.subj_name 
HAVING count(sl.lecturer_id) > :count_1 ORDER BY su.subj_name

5


[('Английский язык',),
 ('Информатика',),
 ('История',),
 ('ОБЖ',),
 ('Программирование на языках Python и SQL',)]

<img src="../Img/Python_SQL_PI.png" style="width:100%">


<p style="font-size:22px;text-align:center">Семинар 5.1. Соединение таблиц. Оператор JOIN</p> 

<a id = Home></a>

**11 апреля 2022 года, пн**<br>
10:10 - 11:40<br>
Щ38/509(кк) (ул. Щербаковская, 38)<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (4 подгруппа)<br>

11:50 - 13:20<br>
Щ38/509(кк) (ул. Щербаковская, 38)<br>
Поток:ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (3 подгруппа)


**14 апреля 2022 года, чт**<br>
17:20 - 18:50<br>
В4/3411(кк) (4-й Вешняковский проезд, 4)<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (2 подгруппа)<br>


**21 апреля 2022 года, чт**<br>
18:55 - 20:25<br>
В4/3411(кк) (4-й Вешняковский проезд, 4)<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (1 подгруппа)<br>

<p style="font-size:22px;text-align:center">Семинар 5.2. Соединение таблиц. Оператор JOIN</p>

**14 апреля 2022 года, чт**<br>
18:55 - 20:25<br>
В4/3411(кк) (4-й Вешняковский проезд, 4)<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (2 подгруппа)<br>

**18 апреля 2022 года, пн**<br>
10:10 - 11:40<br>
Щ38/509(кк) (ул. Щербаковская, 38)<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (3 подгруппа)<br>

11:50 - 13:20<br>
Щ38/509(кк) (ул. Щербаковская, 38)<br>
Поток:ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (4 подгруппа)<br>

**28 апреля 2022 года, чт**<br>
17:20 - 18:50<br>
В4/3411(кк) (4-й Вешняковский проезд, 4)<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (1 подгруппа)<br>


```
STUDENT
STUDENT_ID — числовой код, идентифицирующий студента (идентификатор студента),
SURNAME — фамилия студента,
NAME — имя студента,
STIPEND — стипендия, которую получает студент,
KURS — курс, на котором учится студент,
CITY — город, в котором живет студент,
BIRTHDAY — дата рождения студента,
UNIV_ID — идентификатор университета, в котором учится студент.


LECTURER
LECTURER_ID — идентификатор преподавателя,
SURNAME — фамилия преподавателя,
NAME — имя преподавателя,
CITY — город, в котором живет преподаватель,
UNIV_ID — идентификатор университета, в котором работает преподаватель


SUBJECT
SUBJ_ID — идентификатор предмета обучения,
SUBJ_NAME — наименование предмета обучения,
HOUR — количество часов, отводимых на изучение предмета,
SEMESTER — семестр, в котором изучается данный предмет.


UNIVERSITY
UNIV_ID — идентификатор университета,
UNIV_NAME — название университета,
RATING — рейтинг университета,
CITY — город, в котором расположен университет.


EXAM_MARKS
EXAM_ID — идентификатор экзамена,
STUDENT_ID — идентификатор студента,
SUBJ_ID — идентификатор предмета обучения,
MARK — экзаменационная оценка,
EXAM_DATE — дата экзамена.


SUBJ_LECT
LECTURER_ID — идентификатор преподавателя,
SUBJ_ID — идентификатор предмета обучения.
```

<img src="../Img/Students_Schema.png" style="width:60%">